In [1]:
#######################################
#!/usr/bin/python3.8
# -*- coding: utf-8 -*-
# @author: chenzh05
# @Desc: { notion api implementation }
# @Date: 2023/08/09 15:29
#######################################
# notion-api: https://developers.notion.com/reference/intro

import requests
import json


def algnprint(input_dict):
    max_key_length = max(len(key) for key in input_dict.keys()) + 1
    for key, value in input_dict.items():
        key += ":"
        formatted_key = f"{key:{max_key_length}}"
        print(f"{formatted_key} {value}")


def jprint(dict):
    print(json.dumps(dict, indent=4, ensure_ascii=False))


def jsave(dict, dir=None):
    with open(dir, "w+") as f:
        json.dump(dict, f, indent=4, ensure_ascii=False)
    print(f"jsave to {dir}")


class Config:
    token = "secret_s6zhSzRMCv1s9zf9rwPLbd6SVc5H9656QCGyWgd4s1F"
    endpoint = "https://api.notion.com/v1"
    user_id = "5b02b983-2356-4012-b7ff-bea7da21ef08"
    workspace_name = "Jo's Notion"

    db_inbox = "48de1468a76942dc9c93128fc56babb1"
    db_daily_paper = "08a9b64c5d344a72967cd3bfdf8f8368"
    db_book = "61abd05763c84033a98f577e7e92ad16"
    db_test = "e04e534cad4a44c5b1c0c11f869f0197"


def notion_api(method, route, headers=None, data=None):
    """
    General API call template to notion
    """
    url = Config.endpoint + route
    if not headers:
        headers = {
            "Authorization": f"Bearer {Config.token}",
            "Notion-Version": "2022-06-28",
            "Content-Type": "application/json",
        }
    response = requests.request(method, url, headers=headers, json=data)
    return response


def database_retireve(database_id):
    """
    Retireve a database from notion with database_id
    """
    method = "GET"
    route = f"/databases/{database_id}/"
    response = notion_api(method=method, route=route)
    response.raise_for_status()
    return response


def database_query(database_id):
    """
    Query a database from notion with database_id
    """
    method = "POST"
    route = f"/databases/{database_id}/query"
    response = notion_api(method=method, route=route)
    response.raise_for_status()
    return response


def page_create(parent, properties, children=None, icon=None, cover=None):
    """
    create a new page attached either to a page or a database
    """
    method = "POST"
    route = f"/pages"
    data = {
        "parent": parent,
        "properties": properties,
        "children": children,
        "icon": icon,
        "cover": cover,
    }
    response = notion_api(method=method, route=route, data=data)
    response.raise_for_status()
    return response

def database_get_properties(database_id):
    response = database_retireve(database_id)
    properties = response.json()["properties"]
    return properties


In [50]:
#######################################
#!/usr/bin/python3.8
# -*- coding: utf-8 -*-
# @author: chenzh05
# @Desc: { notion api request body template }
# @Date: 2023/08/09 16:15
#######################################

def tmpl_page_property(property: dict, **kwargs) -> dict:
    _type = property["type"]

    if _type == "checkbox":
        checked = kwargs["checked"] if "checked" in kwargs else None
        data = {
            # "id": property["id"],
            # "type": "checkbox",
            "checkbox": checked,
        }

    if _type == "date":
        start = kwargs["start"] if "start" in kwargs else None
        end = kwargs["end"] if "end" in kwargs else None
        time_zone = kwargs["time_zone"] if "time_zone" in kwargs else None
        data = {
            # "id": property["id"],
            # "type": "date",
            "date": {"start": start, "end": end, "time_zone": time_zone},
        }

    if _type == "multi_select":
        # options = property["multi_select"]["options"]
        # options = [i["name"] for i in range(len(options))]
        options = kwargs["tags"] if "tags" in kwargs else None
        options = [{"name": i} for i in options]
        data = {
            # "id": property["id"],
            # "type": "multi_select",
            "multi_select": options,
        }

    if _type == "title":
        content = kwargs["title"] if "title" in kwargs else None
        link = kwargs["link"] if "link" in kwargs else None
        data = {
            # "id": "title",
            # "type": "title",
            "title": [{"text": {"content": content}}],
        }

    # temp = {}
    # temp[property["name"]] = data
    return data


def tmpl_rich_text(
    content: str,
    link: bool = None,
    bold: bool = False,
    italic: bool = False,
    strikethrough: bool = False,
    underline: bool = False,
    code: bool = False,
    color: str = "default",
):
    tmpl = {
        "type": "text",
        "text": {"content": content, "link": link},
        "annotations": tmpl_annotation(
            bold, italic, strikethrough, underline, code, color
        ),
        "plain_text": content,
        "href": link,
    }

    return tmpl


def tmpl_annotation(
    bold=False,
    italic=False,
    strikethrough=False,
    underline=False,
    code=False,
    color="default",
):
    tmpl = {
        "bold": bold,
        "italic": italic,
        "strikethrough": strikethrough,
        "underline": underline,
        "code": code,
        "color": color,
    }
    return tmpl


In [3]:
response = database_get_properties(Config.db_test)

In [7]:
algnprint({k:v["type"] for  k,v in response.items()})

pblished:     date
Status:       status
Created time: created_time
Text:         rich_text
Tags:         multi_select
URL:          url
money:        number
Select:       select
Checkbox:     checkbox
title:        title


In [69]:
property = {}

In [70]:
# property['Checkbox'] = tmpl_page_property(response['Checkbox'],checked=True)
# property['pblished'] = tmpl_page_property(response['pblished'],start="2222-2-24")
# property['Tags'] = tmpl_page_property(response['Tags'],tags=['cs','nlp'])
property['title'] = tmpl_page_property(response['title'],title='test2222222222222')

In [71]:
jprint(property)

{
    "title": {
        "title": [
            {
                "text": {
                    "content": "test2222222222222"
                }
            }
        ]
    }
}


In [72]:
parent = {"database_id": Config.db_test}
response0 = page_create(parent, property)

HTTPError: 400 Client Error: Bad Request for url: https://api.notion.com/v1/pages